In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/LangChain')
with open('env_vars.txt', 'r') as file:
  for line in file:
    key, value = line.strip().split('=')
    os.environ[key] = value

Mounted at /content/drive


In [ ]:
!pip install boto3

https://github.com/langchain-ai/langchain/issues/6917

In [6]:
import boto3

# Get the service resource.
dynamodb = boto3.resource("dynamodb")

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

# Print out some data about the table.
print(table.item_count)

0


In [ ]:
!pip install langchain==0.0.247 openai

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain



llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
message_history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="0")
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history, return_messages=True
)
bot = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [12]:
bot.run({"question": "hi I'm Nick"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Nick

> Finished chain.


'Hello Nick! How can I assist you today?'

In [13]:
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="0")
history.messages

[HumanMessage(content="hi I'm Nick", additional_kwargs={}, example=False),
 AIMessage(content='Hello Nick! How can I assist you today?', additional_kwargs={}, example=False)]

In [15]:
def bot(session_id):
  llm = ChatOpenAI()
  prompt = ChatPromptTemplate(
      messages=[
          SystemMessagePromptTemplate.from_template(
              "You are a nice chatbot having a conversation with a human."
          ),
          MessagesPlaceholder(variable_name="chat_history"),
          HumanMessagePromptTemplate.from_template("{question}")
      ]
  )
  message_history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id=session_id)
  memory = ConversationBufferMemory(
      memory_key="chat_history", chat_memory=message_history, return_messages=True
  )
  bot = LLMChain(
      llm=llm,
      prompt=prompt,
      verbose=True,
      memory=memory
  )
  return bot

In [16]:
bot("nick").run({"question": "hi I'm Nick"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Nick

> Finished chain.


'Hello Nick! How can I assist you today?'

In [17]:
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="nick")
history.messages

[HumanMessage(content="hi I'm Nick", additional_kwargs={}, example=False),
 AIMessage(content='Hello Nick! How can I assist you today?', additional_kwargs={}, example=False)]

In [18]:
bot("oliver").run({"question": "hi I'm Oliver"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Oliver

> Finished chain.


'Hello Oliver! How can I assist you today?'

In [19]:
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="oliver")
history.messages

[HumanMessage(content="hi I'm Oliver", additional_kwargs={}, example=False),
 AIMessage(content='Hello Oliver! How can I assist you today?', additional_kwargs={}, example=False)]

In [20]:
bot("nick").run({"question": "我的名字是什麼"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Nick
AI: Hello Nick! How can I assist you today?
Human: 我的名字是什麼

> Finished chain.


'你剛才說你的名字是Nick，是嗎？那麼你的名字就是Nick。你有什麼需要幫忙的嗎？'

In [21]:
bot("oliver").run({"question": "我的名字是什麼"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Oliver
AI: Hello Oliver! How can I assist you today?
Human: 我的名字是什麼

> Finished chain.


'你的名字是Oliver，不是嗎？'